<a href="https://colab.research.google.com/github/IllgamhoDuck/Quantum-Circuit-Optimization-with-Deep-learning/blob/master/predicting_statevectors_from_circuits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Use Neural Networks to imitate Quantum Computers with Qiskit

Quantum computers are rare, and even simulating quantum computation is expensive. With neural networks you can simulate quantum computers. Qiskit provides many useful functions along the way.

## Predicting Statevectors from Quantum Circuits

You can predict statevectors from quantum circuits using neural networks. 

You can see in this [Google Sheets](https://docs.google.com/spreadsheets/d/1LPAHmYtP5d9_qKSbEQmxlSk6NTAQU2ytRe82e3c6dVQ/edit?usp=sharing) spreadsheet an example of how we ran multiple experiments to find the best way to predict statevectors from quantum circuits.




When we ran the experiments, we found it interesting that the neural networks were able to predict the statevectors much better when they were trained on more data. The RMSE (a way to measure how different the predicted and actual statevectors are) for totally random predictions is 600, while for perfect predictions it would be 0. With 1k data, the RMSE is 320. With 5m data, the RMSE is 120 and with 8m data the RMSE is 70. 

First we take the 10 lines of circuit-statevector pairs we generated earlier but write them in json format.

In [0]:
%%writefile 10_quantum_circuits.json
[{"statevector": {"state_11111": 0, "state_10111": 0, "state_11011": 0, "state_11100": 0, "state_01001": 0, "state_01100": 0, "state_00000": 0, "state_00100": 0, "state_01000": 0, "state_11110": 0, "state_11000": 0, "state_00001": 0, "state_00111": 0, "state_00010": 0, "state_11010": 0, "state_10000": 1024, "state_10110": 0, "state_10010": 0, "state_00101": 0, "state_10100": 0, "state_01111": 0, "state_11001": 0, "state_10001": 0, "state_00011": 0, "state_10101": 0, "state_01010": 0, "state_11101": 0, "state_10011": 0, "state_01110": 0, "state_01101": 0, "state_00110": 0, "state_01011": 0}, "gate_11": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 0, "Angle_1": -1, "Target": 1, "Angle_2": -1, "Gate_Number": 11}, "gate_30": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_28": {"Gate_Type": "Measure", "Angle_3": -1, "Control": 3, "Angle_1": -1, "Target": -1, "Angle_2": -1, "Gate_Number": 28}, "gate_08": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 0, "Angle_1": -1, "Target": 1, "Angle_2": -1, "Gate_Number": 8}, "gate_12": {"Gate_Type": "U3Gate", "Angle_3": 1.0, "Control": 3, "Angle_1": 0.5, "Target": -1, "Angle_2": 0.0, "Gate_Number": 12}, "gate_20": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 3, "Angle_1": -1, "Target": 2, "Angle_2": -1, "Gate_Number": 20}, "gate_15": {"Gate_Type": "U3Gate", "Angle_3": 0.5, "Control": 4, "Angle_1": 1.0, "Target": -1, "Angle_2": 0.5, "Gate_Number": 15}, "gate_32": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_23": {"Gate_Type": "U3Gate", "Angle_3": 0.0, "Control": 2, "Angle_1": 0.8049289612268833, "Target": -1, "Angle_2": 0.8607349082288277, "Gate_Number": 23}, "gate_02": {"Gate_Type": "U3Gate", "Angle_3": 1.0, "Control": 0, "Angle_1": 0.5, "Target": -1, "Angle_2": 0.0, "Gate_Number": 2}, "gate_37": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_31": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_10": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 1, "Angle_1": -1, "Target": 0, "Angle_2": -1, "Gate_Number": 10}, "gate_17": {"Gate_Type": "U3Gate", "Angle_3": -0.7136315159360238, "Control": 2, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 17}, "gate_01": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 2, "Angle_1": -1, "Target": 0, "Angle_2": -1, "Gate_Number": 1}, "gate_39": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_00": {"Gate_Type": "U3Gate", "Angle_3": 1.0, "Control": 0, "Angle_1": 0.5, "Target": -1, "Angle_2": 0.0, "Gate_Number": 0}, "gate_40": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_05": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 1, "Angle_1": -1, "Target": 3, "Angle_2": -1, "Gate_Number": 5}, "gate_16": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 4, "Angle_1": -1, "Target": 2, "Angle_2": -1, "Gate_Number": 16}, "gate_18": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 4, "Angle_1": -1, "Target": 2, "Angle_2": -1, "Gate_Number": 18}, "gate_27": {"Gate_Type": "Measure", "Angle_3": -1, "Control": 2, "Angle_1": -1, "Target": -1, "Angle_2": -1, "Gate_Number": 27}, "gate_25": {"Gate_Type": "Measure", "Angle_3": -1, "Control": 0, "Angle_1": -1, "Target": -1, "Angle_2": -1, "Gate_Number": 25}, "gate_34": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_35": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_06": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 0, "Angle_1": -1, "Target": 1, "Angle_2": -1, "Gate_Number": 6}, "gate_29": {"Gate_Type": "Measure", "Angle_3": -1, "Control": 4, "Angle_1": -1, "Target": -1, "Angle_2": -1, "Gate_Number": 29}, "gate_38": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_19": {"Gate_Type": "U3Gate", "Angle_3": -0.1511817108902763, "Control": 2, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 19}, "QASM_File_Name": "3125_random_5_qubit_circuits/circuit_1033.qasm", "gate_33": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_09": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 0, "Angle_1": -1, "Target": 1, "Angle_2": -1, "Gate_Number": 9}, "gate_24": {"Gate_Type": "U3Gate", "Angle_3": -0.5, "Control": 4, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 24}, "gate_13": {"Gate_Type": "U3Gate", "Angle_3": 0.7428302918802038, "Control": 3, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 13}, "gate_14": {"Gate_Type": "U3Gate", "Angle_3": 0.7095531973385508, "Control": 3, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 14}, "gate_26": {"Gate_Type": "Measure", "Angle_3": -1, "Control": 1, "Angle_1": -1, "Target": -1, "Angle_2": -1, "Gate_Number": 26}, "gate_07": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 1, "Angle_1": -1, "Target": 0, "Angle_2": -1, "Gate_Number": 7}, "gate_03": {"Gate_Type": "U3Gate", "Angle_3": 0.7136315159360238, "Control": 2, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 3}, "gate_21": {"Gate_Type": "U3Gate", "Angle_3": -0.7095531973385508, "Control": 2, "Angle_1": -0.8049289612268833, "Target": -1, "Angle_2": 0.0, "Gate_Number": 21}, "gate_22": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 3, "Angle_1": -1, "Target": 2, "Angle_2": -1, "Gate_Number": 22}, "gate_36": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_04": {"Gate_Type": "U3Gate", "Angle_3": 1.0, "Control": 3, "Angle_1": 0.5, "Target": -1, "Angle_2": 0.0, "Gate_Number": 4}},
{"statevector": {"state_11111": 0, "state_10111": 0, "state_11011": 0, "state_11100": 1, "state_01001": 0, "state_01100": 7, "state_00000": 0, "state_00100": 6, "state_01000": 0, "state_11110": 0, "state_11000": 0, "state_00001": 0, "state_00111": 0, "state_00010": 0, "state_11010": 0, "state_10000": 0, "state_10110": 0, "state_10010": 0, "state_00101": 276, "state_10100": 0, "state_01111": 0, "state_11001": 0, "state_10001": 0, "state_00011": 0, "state_10101": 70, "state_01010": 0, "state_11101": 126, "state_10011": 0, "state_01110": 0, "state_01101": 538, "state_00110": 0, "state_01011": 0}, "gate_11": {"Gate_Type": "U3Gate", "Angle_3": 1.0, "Control": 2, "Angle_1": 1.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 11}, "gate_30": {"Gate_Type": "U3Gate", "Angle_3": 0.5, "Control": 4, "Angle_1": 0.712385061802925, "Target": -1, "Angle_2": -0.5, "Gate_Number": 30}, "gate_28": {"Gate_Type": "U3Gate", "Angle_3": 0.5, "Control": 4, "Angle_1": 1.0, "Target": -1, "Angle_2": 0.5, "Gate_Number": 28}, "gate_08": {"Gate_Type": "U3Gate", "Angle_3": 0.303869713933786, "Control": 2, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 8}, "gate_12": {"Gate_Type": "U3Gate", "Angle_3": 1.0, "Control": 3, "Angle_1": 0.5, "Target": -1, "Angle_2": 0.0, "Gate_Number": 12}, "gate_20": {"Gate_Type": "U3Gate", "Angle_3": 1.1090761541508274, "Control": 1, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 20}, "gate_15": {"Gate_Type": "U3Gate", "Angle_3": 1.0, "Control": 3, "Angle_1": 0.5, "Target": -1, "Angle_2": 0.0, "Gate_Number": 15}, "gate_32": {"Gate_Type": "Measure", "Angle_3": -1, "Control": 1, "Angle_1": -1, "Target": -1, "Angle_2": -1, "Gate_Number": 32}, "gate_23": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 1, "Angle_1": -1, "Target": 0, "Angle_2": -1, "Gate_Number": 23}, "gate_02": {"Gate_Type": "U3Gate", "Angle_3": 0.0, "Control": 0, "Angle_1": 0.9137505906370721, "Target": -1, "Angle_2": 0.0, "Gate_Number": 2}, "gate_37": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_31": {"Gate_Type": "Measure", "Angle_3": -1, "Control": 0, "Angle_1": -1, "Target": -1, "Angle_2": -1, "Gate_Number": 31}, "gate_10": {"Gate_Type": "U3Gate", "Angle_3": 0.5, "Control": 2, "Angle_1": 1.0, "Target": -1, "Angle_2": 0.5, "Gate_Number": 10}, "gate_17": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 1, "Angle_1": -1, "Target": 3, "Angle_2": -1, "Gate_Number": 17}, "gate_01": {"Gate_Type": "U3Gate", "Angle_3": 1.0, "Control": 0, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 1}, "gate_39": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_00": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 1, "Angle_1": -1, "Target": 0, "Angle_2": -1, "Gate_Number": 0}, "gate_40": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_05": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 3, "Angle_1": -1, "Target": 2, "Angle_2": -1, "Gate_Number": 5}, "gate_16": {"Gate_Type": "U3Gate", "Angle_3": -0.25090112293232186, "Control": 3, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 16}, "gate_18": {"Gate_Type": "U3Gate", "Angle_3": -1.0885938102646706, "Control": 3, "Angle_1": -0.40406205203081336, "Target": -1, "Angle_2": 0.0, "Gate_Number": 18}, "gate_27": {"Gate_Type": "U3Gate", "Angle_3": -0.25, "Control": 4, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 27}, "gate_25": {"Gate_Type": "U3Gate", "Angle_3": 0.0, "Control": 3, "Angle_1": 0.40406205203081336, "Target": -1, "Angle_2": 1.3394949331969916, "Gate_Number": 25}, "gate_34": {"Gate_Type": "Measure", "Angle_3": -1, "Control": 3, "Angle_1": -1, "Target": -1, "Angle_2": -1, "Gate_Number": 34}, "gate_35": {"Gate_Type": "Measure", "Angle_3": -1, "Control": 4, "Angle_1": -1, "Target": -1, "Angle_2": -1, "Gate_Number": 35}, "gate_06": {"Gate_Type": "U3Gate", "Angle_3": -0.303869713933786, "Control": 2, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 6}, "gate_29": {"Gate_Type": "U3Gate", "Angle_3": 1.413511704425418, "Control": 4, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 29}, "gate_38": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_19": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 1, "Angle_1": -1, "Target": 3, "Angle_2": -1, "Gate_Number": 19}, "QASM_File_Name": "3125_random_5_qubit_circuits/circuit_1093.qasm", "gate_33": {"Gate_Type": "Measure", "Angle_3": -1, "Control": 2, "Angle_1": -1, "Target": -1, "Angle_2": -1, "Gate_Number": 33}, "gate_09": {"Gate_Type": "U3Gate", "Angle_3": 1.0, "Control": 2, "Angle_1": 1.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 9}, "gate_24": {"Gate_Type": "U3Gate", "Angle_3": 0.0, "Control": 0, "Angle_1": 0.7353563437579959, "Target": -1, "Angle_2": 0.32998262752760155, "Gate_Number": 24}, "gate_13": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 1, "Angle_1": -1, "Target": 3, "Angle_2": -1, "Gate_Number": 13}, "gate_14": {"Gate_Type": "U3Gate", "Angle_3": 1.0885938102646706, "Control": 1, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 14}, "gate_26": {"Gate_Type": "U3Gate", "Angle_3": 0.5, "Control": 3, "Angle_1": 1.3927355704625703, "Target": -1, "Angle_2": -0.5, "Gate_Number": 26}, "gate_07": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 3, "Angle_1": -1, "Target": 2, "Angle_2": -1, "Gate_Number": 7}, "gate_03": {"Gate_Type": "U3Gate", "Angle_3": 0.7790935266232226, "Control": 0, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 3}, "gate_21": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 1, "Angle_1": -1, "Target": 0, "Angle_2": -1, "Gate_Number": 21}, "gate_22": {"Gate_Type": "U3Gate", "Angle_3": -1.1090761541508274, "Control": 0, "Angle_1": -0.7353563437579959, "Target": -1, "Angle_2": 0.0, "Gate_Number": 22}, "gate_36": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_04": {"Gate_Type": "U3Gate", "Angle_3": 0.303869713933786, "Control": 3, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 4}},
{"statevector": {"state_11111": 0, "state_10111": 0, "state_11011": 0, "state_11100": 0, "state_01001": 473, "state_01100": 0, "state_00000": 0, "state_00100": 0, "state_01000": 0, "state_11110": 0, "state_11000": 0, "state_00001": 0, "state_00111": 0, "state_00010": 0, "state_11010": 0, "state_10000": 0, "state_10110": 0, "state_10010": 0, "state_00101": 0, "state_10100": 0, "state_01111": 0, "state_11001": 0, "state_10001": 0, "state_00011": 507, "state_10101": 0, "state_01010": 0, "state_11101": 0, "state_10011": 0, "state_01110": 0, "state_01101": 0, "state_00110": 0, "state_01011": 44}, "gate_11": {"Gate_Type": "U3Gate", "Angle_3": -0.5846280370467051, "Control": 2, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 11}, "gate_30": {"Gate_Type": "U3Gate", "Angle_3": 0.5, "Control": 4, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 30}, "gate_28": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 0, "Angle_1": -1, "Target": 2, "Angle_2": -1, "Gate_Number": 28}, "gate_08": {"Gate_Type": "U3Gate", "Angle_3": 1.8516227649410237, "Control": 1, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 8}, "gate_12": {"Gate_Type": "U3Gate", "Angle_3": 0.7700503795647785, "Control": 3, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 12}, "gate_20": {"Gate_Type": "U3Gate", "Angle_3": -0.5, "Control": 4, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 20}, "gate_15": {"Gate_Type": "U3Gate", "Angle_3": 1.7796287299937472, "Control": 3, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 15}, "gate_32": {"Gate_Type": "Measure", "Angle_3": -1, "Control": 0, "Angle_1": -1, "Target": -1, "Angle_2": -1, "Gate_Number": 32}, "gate_23": {"Gate_Type": "U3Gate", "Angle_3": -1.2442257118795548, "Control": 2, "Angle_1": -0.5165480413989794, "Target": -1, "Angle_2": 0.0, "Gate_Number": 23}, "gate_02": {"Gate_Type": "U3Gate", "Angle_3": 0.0, "Control": 1, "Angle_1": 0.9791406541201794, "Target": -1, "Angle_2": 0.0, "Gate_Number": 2}, "gate_37": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_31": {"Gate_Type": "U3Gate", "Angle_3": 1.9914471621193017, "Control": 4, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 31}, "gate_10": {"Gate_Type": "U3Gate", "Angle_3": -0.25, "Control": 2, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 10}, "gate_17": {"Gate_Type": "U3Gate", "Angle_3": -1.7796287299937472, "Control": 1, "Angle_1": -0.6005123411564115, "Target": -1, "Angle_2": 0.0, "Gate_Number": 17}, "gate_01": {"Gate_Type": "U3Gate", "Angle_3": 0.8807997938322562, "Control": 0, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 1}, "gate_39": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_00": {"Gate_Type": "U3Gate", "Angle_3": -0.5, "Control": 0, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 0}, "gate_40": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_05": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 0, "Angle_1": -1, "Target": 1, "Angle_2": -1, "Gate_Number": 5}, "gate_16": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 3, "Angle_1": -1, "Target": 1, "Angle_2": -1, "Gate_Number": 16}, "gate_18": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 3, "Angle_1": -1, "Target": 1, "Angle_2": -1, "Gate_Number": 18}, "gate_27": {"Gate_Type": "U3Gate", "Angle_3": 1.0, "Control": 2, "Angle_1": 0.5, "Target": -1, "Angle_2": 0.0, "Gate_Number": 27}, "gate_25": {"Gate_Type": "U3Gate", "Angle_3": 0.0, "Control": 2, "Angle_1": 0.5165480413989794, "Target": -1, "Angle_2": 1.828853748926263, "Gate_Number": 25}, "gate_34": {"Gate_Type": "Measure", "Angle_3": -1, "Control": 2, "Angle_1": -1, "Target": -1, "Angle_2": -1, "Gate_Number": 34}, "gate_35": {"Gate_Type": "Measure", "Angle_3": -1, "Control": 3, "Angle_1": -1, "Target": -1, "Angle_2": -1, "Gate_Number": 35}, "gate_06": {"Gate_Type": "U3Gate", "Angle_3": 1.0, "Control": 0, "Angle_1": 1.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 6}, "gate_29": {"Gate_Type": "U3Gate", "Angle_3": 1.0, "Control": 2, "Angle_1": 0.5, "Target": -1, "Angle_2": 0.0, "Gate_Number": 29}, "gate_38": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_19": {"Gate_Type": "U3Gate", "Angle_3": 0.0, "Control": 1, "Angle_1": 0.6005123411564115, "Target": -1, "Angle_2": 1.9488056187580274, "Gate_Number": 19}, "QASM_File_Name": "3125_random_5_qubit_circuits/circuit_2511.qasm", "gate_33": {"Gate_Type": "Measure", "Angle_3": -1, "Control": 1, "Angle_1": -1, "Target": -1, "Angle_2": -1, "Gate_Number": 33}, "gate_09": {"Gate_Type": "U3Gate", "Angle_3": -0.169176888764282, "Control": 1, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 9}, "gate_24": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 4, "Angle_1": -1, "Target": 2, "Angle_2": -1, "Gate_Number": 24}, "gate_13": {"Gate_Type": "U3Gate", "Angle_3": 0.4531223252373775, "Control": 3, "Angle_1": 0.5, "Target": -1, "Angle_2": 0.6151476265908303, "Gate_Number": 13}, "gate_14": {"Gate_Type": "U3Gate", "Angle_3": 1.0, "Control": 3, "Angle_1": 1.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 14}, "gate_26": {"Gate_Type": "U3Gate", "Angle_3": -0.5, "Control": 2, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 26}, "gate_07": {"Gate_Type": "U3Gate", "Angle_3": 0.8807997938322562, "Control": 1, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 7}, "gate_03": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 0, "Angle_1": -1, "Target": 1, "Angle_2": -1, "Gate_Number": 3}, "gate_21": {"Gate_Type": "U3Gate", "Angle_3": 1.2442257118795548, "Control": 4, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 21}, "gate_22": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 4, "Angle_1": -1, "Target": 2, "Angle_2": -1, "Gate_Number": 22}, "gate_36": {"Gate_Type": "Measure", "Angle_3": -1, "Control": 4, "Angle_1": -1, "Target": -1, "Angle_2": -1, "Gate_Number": 36}, "gate_04": {"Gate_Type": "U3Gate", "Angle_3": -0.8807997938322562, "Control": 1, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 4}},
{"statevector": {"state_11111": 0, "state_10111": 0, "state_11011": 0, "state_11100": 0, "state_01001": 0, "state_01100": 0, "state_00000": 0, "state_00100": 0, "state_01000": 0, "state_11110": 0, "state_11000": 0, "state_00001": 0, "state_00111": 1, "state_00010": 0, "state_11010": 0, "state_10000": 0, "state_10110": 0, "state_10010": 0, "state_00101": 0, "state_10100": 0, "state_01111": 0, "state_11001": 0, "state_10001": 0, "state_00011": 0, "state_10101": 0, "state_01010": 0, "state_11101": 0, "state_10011": 0, "state_01110": 0, "state_01101": 0, "state_00110": 1023, "state_01011": 0}, "gate_11": {"Gate_Type": "U3Gate", "Angle_3": 0.1028622995855569, "Control": 4, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 11}, "gate_30": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_28": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_08": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 1, "Angle_1": -1, "Target": 4, "Angle_2": -1, "Gate_Number": 8}, "gate_12": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 2, "Angle_1": -1, "Target": 4, "Angle_2": -1, "Gate_Number": 12}, "gate_20": {"Gate_Type": "U3Gate", "Angle_3": -0.5989071275785603, "Control": 4, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 20}, "gate_15": {"Gate_Type": "U3Gate", "Angle_3": -0.5, "Control": 2, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 15}, "gate_32": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_23": {"Gate_Type": "Measure", "Angle_3": -1, "Control": 1, "Angle_1": -1, "Target": -1, "Angle_2": -1, "Gate_Number": 23}, "gate_02": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 3, "Angle_1": -1, "Target": 2, "Angle_2": -1, "Gate_Number": 2}, "gate_37": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_31": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_10": {"Gate_Type": "U3Gate", "Angle_3": 1.0, "Control": 4, "Angle_1": 0.5, "Target": -1, "Angle_2": 0.0, "Gate_Number": 10}, "gate_17": {"Gate_Type": "U3Gate", "Angle_3": 0.5, "Control": 2, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 17}, "gate_01": {"Gate_Type": "U3Gate", "Angle_3": 0.5, "Control": 0, "Angle_1": 1.9861418429793378, "Target": -1, "Angle_2": -0.5, "Gate_Number": 1}, "gate_39": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_00": {"Gate_Type": "U3Gate", "Angle_3": 1.0, "Control": 0, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 0}, "gate_40": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_05": {"Gate_Type": "U3Gate", "Angle_3": -0.25, "Control": 3, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 5}, "gate_16": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 1, "Angle_1": -1, "Target": 2, "Angle_2": -1, "Gate_Number": 16}, "gate_18": {"Gate_Type": "U3Gate", "Angle_3": 0.5989071275785603, "Control": 4, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 18}, "gate_27": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_25": {"Gate_Type": "Measure", "Angle_3": -1, "Control": 3, "Angle_1": -1, "Target": -1, "Angle_2": -1, "Gate_Number": 25}, "gate_34": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_35": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_06": {"Gate_Type": "U3Gate", "Angle_3": 0.5, "Control": 3, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 6}, "gate_29": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_38": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_19": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 0, "Angle_1": -1, "Target": 4, "Angle_2": -1, "Gate_Number": 19}, "QASM_File_Name": "3125_random_5_qubit_circuits/circuit_998.qasm", "gate_33": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_09": {"Gate_Type": "U3Gate", "Angle_3": 0.5, "Control": 1, "Angle_1": 1.0, "Target": -1, "Angle_2": 0.5, "Gate_Number": 9}, "gate_24": {"Gate_Type": "Measure", "Angle_3": -1, "Control": 2, "Angle_1": -1, "Target": -1, "Angle_2": -1, "Gate_Number": 24}, "gate_13": {"Gate_Type": "U3Gate", "Angle_3": -0.1028622995855569, "Control": 4, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 13}, "gate_14": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 2, "Angle_1": -1, "Target": 4, "Angle_2": -1, "Gate_Number": 14}, "gate_26": {"Gate_Type": "Measure", "Angle_3": -1, "Control": 4, "Angle_1": -1, "Target": -1, "Angle_2": -1, "Gate_Number": 26}, "gate_07": {"Gate_Type": "U3Gate", "Angle_3": 1.0, "Control": 4, "Angle_1": 0.5, "Target": -1, "Angle_2": 0.0, "Gate_Number": 7}, "gate_03": {"Gate_Type": "U3Gate", "Angle_3": 0.3683039947093093, "Control": 2, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 3}, "gate_21": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 0, "Angle_1": -1, "Target": 4, "Angle_2": -1, "Gate_Number": 21}, "gate_22": {"Gate_Type": "Measure", "Angle_3": -1, "Control": 0, "Angle_1": -1, "Target": -1, "Angle_2": -1, "Gate_Number": 22}, "gate_36": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_04": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 3, "Angle_1": -1, "Target": 2, "Angle_2": -1, "Gate_Number": 4}},
{"statevector": {"state_11111": 0, "state_10111": 0, "state_11011": 0, "state_11100": 0, "state_01001": 0, "state_01100": 0, "state_00000": 1024, "state_00100": 0, "state_01000": 0, "state_11110": 0, "state_11000": 0, "state_00001": 0, "state_00111": 0, "state_00010": 0, "state_11010": 0, "state_10000": 0, "state_10110": 0, "state_10010": 0, "state_00101": 0, "state_10100": 0, "state_01111": 0, "state_11001": 0, "state_10001": 0, "state_00011": 0, "state_10101": 0, "state_01010": 0, "state_11101": 0, "state_10011": 0, "state_01110": 0, "state_01101": 0, "state_00110": 0, "state_01011": 0}, "gate_11": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 2, "Angle_1": -1, "Target": 4, "Angle_2": -1, "Gate_Number": 11}, "gate_30": {"Gate_Type": "U3Gate", "Angle_3": 1.9505773075083912, "Control": 4, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 30}, "gate_28": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 2, "Angle_1": -1, "Target": 1, "Angle_2": -1, "Gate_Number": 28}, "gate_08": {"Gate_Type": "U3Gate", "Angle_3": -0.34754072386077195, "Control": 4, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 8}, "gate_12": {"Gate_Type": "U3Gate", "Angle_3": 0.0, "Control": 4, "Angle_1": 0.711135270090666, "Target": -1, "Angle_2": 1.3320896117497358, "Gate_Number": 12}, "gate_20": {"Gate_Type": "U3Gate", "Angle_3": 1.0, "Control": 4, "Angle_1": 0.5, "Target": -1, "Angle_2": 0.0, "Gate_Number": 20}, "gate_15": {"Gate_Type": "U3Gate", "Angle_3": 0.25, "Control": 4, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 15}, "gate_32": {"Gate_Type": "Measure", "Angle_3": -1, "Control": 0, "Angle_1": -1, "Target": -1, "Angle_2": -1, "Gate_Number": 32}, "gate_23": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 1, "Angle_1": -1, "Target": 4, "Angle_2": -1, "Gate_Number": 23}, "gate_02": {"Gate_Type": "U3Gate", "Angle_3": 0.5, "Control": 0, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 2}, "gate_37": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_31": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 0, "Angle_1": -1, "Target": 4, "Angle_2": -1, "Gate_Number": 31}, "gate_10": {"Gate_Type": "U3Gate", "Angle_3": -0.984548887888964, "Control": 4, "Angle_1": -0.711135270090666, "Target": -1, "Angle_2": 0.0, "Gate_Number": 10}, "gate_17": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 2, "Angle_1": -1, "Target": 3, "Angle_2": -1, "Gate_Number": 17}, "gate_01": {"Gate_Type": "U3Gate", "Angle_3": 1.4475353431734401, "Control": 0, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 1}, "gate_39": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_00": {"Gate_Type": "U3Gate", "Angle_3": 0.25, "Control": 0, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 0}, "gate_40": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_05": {"Gate_Type": "U3Gate", "Angle_3": 1.0, "Control": 3, "Angle_1": 0.5, "Target": -1, "Angle_2": 0.0, "Gate_Number": 5}, "gate_16": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 2, "Angle_1": -1, "Target": 4, "Angle_2": -1, "Gate_Number": 16}, "gate_18": {"Gate_Type": "U3Gate", "Angle_3": -0.25, "Control": 3, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 18}, "gate_27": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 1, "Angle_1": -1, "Target": 2, "Angle_2": -1, "Gate_Number": 27}, "gate_25": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 1, "Angle_1": -1, "Target": 4, "Angle_2": -1, "Gate_Number": 25}, "gate_34": {"Gate_Type": "Measure", "Angle_3": -1, "Control": 2, "Angle_1": -1, "Target": -1, "Angle_2": -1, "Gate_Number": 34}, "gate_35": {"Gate_Type": "Measure", "Angle_3": -1, "Control": 3, "Angle_1": -1, "Target": -1, "Angle_2": -1, "Gate_Number": 35}, "gate_06": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 1, "Angle_1": -1, "Target": 3, "Angle_2": -1, "Gate_Number": 6}, "gate_29": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 1, "Angle_1": -1, "Target": 2, "Angle_2": -1, "Gate_Number": 29}, "gate_38": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_19": {"Gate_Type": "U3Gate", "Angle_3": -0.25, "Control": 4, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 19}, "QASM_File_Name": "3125_random_5_qubit_circuits/circuit_564.qasm", "gate_33": {"Gate_Type": "Measure", "Angle_3": -1, "Control": 1, "Angle_1": -1, "Target": -1, "Angle_2": -1, "Gate_Number": 33}, "gate_09": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 2, "Angle_1": -1, "Target": 4, "Angle_2": -1, "Gate_Number": 9}, "gate_24": {"Gate_Type": "U3Gate", "Angle_3": -0.5314909712084498, "Control": 4, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 24}, "gate_13": {"Gate_Type": "U3Gate", "Angle_3": 0.5, "Control": 4, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 13}, "gate_14": {"Gate_Type": "U3Gate", "Angle_3": 1.0, "Control": 4, "Angle_1": 0.5, "Target": -1, "Angle_2": 0.0, "Gate_Number": 14}, "gate_26": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 0, "Angle_1": -1, "Target": 4, "Angle_2": -1, "Gate_Number": 26}, "gate_07": {"Gate_Type": "U3Gate", "Angle_3": 1.0, "Control": 3, "Angle_1": 0.5, "Target": -1, "Angle_2": 0.0, "Gate_Number": 7}, "gate_03": {"Gate_Type": "U3Gate", "Angle_3": 0.984548887888964, "Control": 2, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 3}, "gate_21": {"Gate_Type": "U3Gate", "Angle_3": -0.5, "Control": 4, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 21}, "gate_22": {"Gate_Type": "U3Gate", "Angle_3": 0.5314909712084498, "Control": 4, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 22}, "gate_36": {"Gate_Type": "Measure", "Angle_3": -1, "Control": 4, "Angle_1": -1, "Target": -1, "Angle_2": -1, "Gate_Number": 36}, "gate_04": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 3, "Angle_1": -1, "Target": 1, "Angle_2": -1, "Gate_Number": 4}},
{"statevector": {"state_11111": 0, "state_10111": 0, "state_11011": 0, "state_11100": 0, "state_01001": 0, "state_01100": 0, "state_00000": 690, "state_00100": 0, "state_01000": 334, "state_11110": 0, "state_11000": 0, "state_00001": 0, "state_00111": 0, "state_00010": 0, "state_11010": 0, "state_10000": 0, "state_10110": 0, "state_10010": 0, "state_00101": 0, "state_10100": 0, "state_01111": 0, "state_11001": 0, "state_10001": 0, "state_00011": 0, "state_10101": 0, "state_01010": 0, "state_11101": 0, "state_10011": 0, "state_01110": 0, "state_01101": 0, "state_00110": 0, "state_01011": 0}, "gate_11": {"Gate_Type": "Measure", "Angle_3": -1, "Control": 0, "Angle_1": -1, "Target": -1, "Angle_2": -1, "Gate_Number": 11}, "gate_30": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_28": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_08": {"Gate_Type": "U3Gate", "Angle_3": -0.12160002696855625, "Control": 2, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 8}, "gate_12": {"Gate_Type": "Measure", "Angle_3": -1, "Control": 1, "Angle_1": -1, "Target": -1, "Angle_2": -1, "Gate_Number": 12}, "gate_20": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_15": {"Gate_Type": "Measure", "Angle_3": -1, "Control": 4, "Angle_1": -1, "Target": -1, "Angle_2": -1, "Gate_Number": 15}, "gate_32": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_23": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_02": {"Gate_Type": "U3Gate", "Angle_3": -0.45279895310823803, "Control": 1, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 2}, "gate_37": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_31": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_10": {"Gate_Type": "U3Gate", "Angle_3": 0.12160002696855625, "Control": 2, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 10}, "gate_17": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_01": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 0, "Angle_1": -1, "Target": 1, "Angle_2": -1, "Gate_Number": 1}, "gate_39": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_00": {"Gate_Type": "U3Gate", "Angle_3": 0.45279895310823803, "Control": 0, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 0}, "gate_40": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_05": {"Gate_Type": "U3Gate", "Angle_3": 0.0, "Control": 3, "Angle_1": 1.6104523376393305, "Target": -1, "Angle_2": 0.0, "Gate_Number": 5}, "gate_16": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_18": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_27": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_25": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_34": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_35": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_06": {"Gate_Type": "U3Gate", "Angle_3": 0.12160002696855625, "Control": 4, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 6}, "gate_29": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_38": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_19": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "QASM_File_Name": "3125_random_5_qubit_circuits/circuit_1224.qasm", "gate_33": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_09": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 4, "Angle_1": -1, "Target": 2, "Angle_2": -1, "Gate_Number": 9}, "gate_24": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_13": {"Gate_Type": "Measure", "Angle_3": -1, "Control": 2, "Angle_1": -1, "Target": -1, "Angle_2": -1, "Gate_Number": 13}, "gate_14": {"Gate_Type": "Measure", "Angle_3": -1, "Control": 3, "Angle_1": -1, "Target": -1, "Angle_2": -1, "Gate_Number": 14}, "gate_26": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_07": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 4, "Angle_1": -1, "Target": 2, "Angle_2": -1, "Gate_Number": 7}, "gate_03": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 0, "Angle_1": -1, "Target": 1, "Angle_2": -1, "Gate_Number": 3}, "gate_21": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_22": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_36": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_04": {"Gate_Type": "U3Gate", "Angle_3": 0.45279895310823803, "Control": 1, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 4}},
{"statevector": {"state_11111": 0, "state_10111": 0, "state_11011": 0, "state_11100": 0, "state_01001": 0, "state_01100": 0, "state_00000": 1024, "state_00100": 0, "state_01000": 0, "state_11110": 0, "state_11000": 0, "state_00001": 0, "state_00111": 0, "state_00010": 0, "state_11010": 0, "state_10000": 0, "state_10110": 0, "state_10010": 0, "state_00101": 0, "state_10100": 0, "state_01111": 0, "state_11001": 0, "state_10001": 0, "state_00011": 0, "state_10101": 0, "state_01010": 0, "state_11101": 0, "state_10011": 0, "state_01110": 0, "state_01101": 0, "state_00110": 0, "state_01011": 0}, "gate_11": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 0, "Angle_1": -1, "Target": 2, "Angle_2": -1, "Gate_Number": 11}, "gate_30": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_28": {"Gate_Type": "Measure", "Angle_3": -1, "Control": 4, "Angle_1": -1, "Target": -1, "Angle_2": -1, "Gate_Number": 28}, "gate_08": {"Gate_Type": "U3Gate", "Angle_3": 0.4584936681031713, "Control": 2, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 8}, "gate_12": {"Gate_Type": "U3Gate", "Angle_3": 0.0, "Control": 2, "Angle_1": 0.17337766460881043, "Target": -1, "Angle_2": 0.20097867929171565, "Gate_Number": 12}, "gate_20": {"Gate_Type": "U3Gate", "Angle_3": -0.25, "Control": 3, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 20}, "gate_15": {"Gate_Type": "U3Gate", "Angle_3": 0.5, "Control": 3, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 15}, "gate_32": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_23": {"Gate_Type": "U3Gate", "Angle_3": 0.25, "Control": 4, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 23}, "gate_02": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 0, "Angle_1": -1, "Target": 1, "Angle_2": -1, "Gate_Number": 2}, "gate_37": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_31": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_10": {"Gate_Type": "U3Gate", "Angle_3": -0.6594723473948892, "Control": 2, "Angle_1": -0.17337766460881043, "Target": -1, "Angle_2": 0.0, "Gate_Number": 10}, "gate_17": {"Gate_Type": "U3Gate", "Angle_3": 1.0, "Control": 3, "Angle_1": 0.5, "Target": -1, "Angle_2": 0.0, "Gate_Number": 17}, "gate_01": {"Gate_Type": "U3Gate", "Angle_3": 0.4541884394143358, "Control": 1, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 1}, "gate_39": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_00": {"Gate_Type": "U3Gate", "Angle_3": 0.9025876956920168, "Control": 0, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 0}, "gate_40": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_05": {"Gate_Type": "U3Gate", "Angle_3": 0.6594723473948892, "Control": 0, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 5}, "gate_16": {"Gate_Type": "U3Gate", "Angle_3": 0.5, "Control": 3, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 16}, "gate_18": {"Gate_Type": "U3Gate", "Angle_3": 0.25, "Control": 3, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 18}, "gate_27": {"Gate_Type": "Measure", "Angle_3": -1, "Control": 3, "Angle_1": -1, "Target": -1, "Angle_2": -1, "Gate_Number": 27}, "gate_25": {"Gate_Type": "Measure", "Angle_3": -1, "Control": 1, "Angle_1": -1, "Target": -1, "Angle_2": -1, "Gate_Number": 25}, "gate_34": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_35": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_06": {"Gate_Type": "U3Gate", "Angle_3": 0.0, "Control": 1, "Angle_1": 0.5499541573692338, "Target": -1, "Angle_2": 0.44839925627767796, "Gate_Number": 6}, "gate_29": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_38": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_19": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 1, "Angle_1": -1, "Target": 3, "Angle_2": -1, "Gate_Number": 19}, "QASM_File_Name": "3125_random_5_qubit_circuits/circuit_450.qasm", "gate_33": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_09": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 0, "Angle_1": -1, "Target": 2, "Angle_2": -1, "Gate_Number": 9}, "gate_24": {"Gate_Type": "Measure", "Angle_3": -1, "Control": 0, "Angle_1": -1, "Target": -1, "Angle_2": -1, "Gate_Number": 24}, "gate_13": {"Gate_Type": "U3Gate", "Angle_3": -0.5, "Control": 3, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 13}, "gate_14": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 4, "Angle_1": -1, "Target": 3, "Angle_2": -1, "Gate_Number": 14}, "gate_26": {"Gate_Type": "Measure", "Angle_3": -1, "Control": 2, "Angle_1": -1, "Target": -1, "Angle_2": -1, "Gate_Number": 26}, "gate_07": {"Gate_Type": "U3Gate", "Angle_3": 1.8972592162875261, "Control": 2, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 7}, "gate_03": {"Gate_Type": "U3Gate", "Angle_3": -0.9025876956920168, "Control": 1, "Angle_1": -0.5499541573692338, "Target": -1, "Angle_2": 0.0, "Gate_Number": 3}, "gate_21": {"Gate_Type": "U3Gate", "Angle_3": 1.0, "Control": 3, "Angle_1": 0.5, "Target": -1, "Angle_2": 0.0, "Gate_Number": 21}, "gate_22": {"Gate_Type": "U3Gate", "Angle_3": -0.5, "Control": 3, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 22}, "gate_36": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_04": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 0, "Angle_1": -1, "Target": 1, "Angle_2": -1, "Gate_Number": 4}},
{"statevector": {"state_11111": 0, "state_10111": 0, "state_11011": 0, "state_11100": 0, "state_01001": 0, "state_01100": 0, "state_00000": 0, "state_00100": 0, "state_01000": 1024, "state_11110": 0, "state_11000": 0, "state_00001": 0, "state_00111": 0, "state_00010": 0, "state_11010": 0, "state_10000": 0, "state_10110": 0, "state_10010": 0, "state_00101": 0, "state_10100": 0, "state_01111": 0, "state_11001": 0, "state_10001": 0, "state_00011": 0, "state_10101": 0, "state_01010": 0, "state_11101": 0, "state_10011": 0, "state_01110": 0, "state_01101": 0, "state_00110": 0, "state_01011": 0}, "gate_11": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 1, "Angle_1": -1, "Target": 0, "Angle_2": -1, "Gate_Number": 11}, "gate_30": {"Gate_Type": "Measure", "Angle_3": -1, "Control": 4, "Angle_1": -1, "Target": -1, "Angle_2": -1, "Gate_Number": 30}, "gate_28": {"Gate_Type": "Measure", "Angle_3": -1, "Control": 2, "Angle_1": -1, "Target": -1, "Angle_2": -1, "Gate_Number": 28}, "gate_08": {"Gate_Type": "U3Gate", "Angle_3": 0.25, "Control": 3, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 8}, "gate_12": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 0, "Angle_1": -1, "Target": 1, "Angle_2": -1, "Gate_Number": 12}, "gate_20": {"Gate_Type": "U3Gate", "Angle_3": 1.0, "Control": 4, "Angle_1": 0.5, "Target": -1, "Angle_2": 0.0, "Gate_Number": 20}, "gate_15": {"Gate_Type": "U3Gate", "Angle_3": 1.0, "Control": 4, "Angle_1": 0.5, "Target": -1, "Angle_2": 0.0, "Gate_Number": 15}, "gate_32": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_23": {"Gate_Type": "U3Gate", "Angle_3": -0.25, "Control": 3, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 23}, "gate_02": {"Gate_Type": "U3Gate", "Angle_3": -0.5, "Control": 3, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 2}, "gate_37": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_31": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_10": {"Gate_Type": "U3Gate", "Angle_3": 1.0183191453400793, "Control": 0, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 10}, "gate_17": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 2, "Angle_1": -1, "Target": 4, "Angle_2": -1, "Gate_Number": 17}, "gate_01": {"Gate_Type": "U3Gate", "Angle_3": 1.3072700714031118, "Control": 1, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 1}, "gate_39": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_00": {"Gate_Type": "U3Gate", "Angle_3": 0.5, "Control": 1, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 0}, "gate_40": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_05": {"Gate_Type": "U3Gate", "Angle_3": 0.5, "Control": 3, "Angle_1": 1.0, "Target": -1, "Angle_2": 0.5, "Gate_Number": 5}, "gate_16": {"Gate_Type": "U3Gate", "Angle_3": 0.25, "Control": 4, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 16}, "gate_18": {"Gate_Type": "U3Gate", "Angle_3": 0.25, "Control": 2, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 18}, "gate_27": {"Gate_Type": "Measure", "Angle_3": -1, "Control": 1, "Angle_1": -1, "Target": -1, "Angle_2": -1, "Gate_Number": 27}, "gate_25": {"Gate_Type": "U3Gate", "Angle_3": -0.5, "Control": 3, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 25}, "gate_34": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_35": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_06": {"Gate_Type": "U3Gate", "Angle_3": 0.5, "Control": 3, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 6}, "gate_29": {"Gate_Type": "Measure", "Angle_3": -1, "Control": 3, "Angle_1": -1, "Target": -1, "Angle_2": -1, "Gate_Number": 29}, "gate_38": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_19": {"Gate_Type": "U3Gate", "Angle_3": -0.25, "Control": 4, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 19}, "QASM_File_Name": "3125_random_5_qubit_circuits/circuit_663.qasm", "gate_33": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_09": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 4, "Angle_1": -1, "Target": 0, "Angle_2": -1, "Gate_Number": 9}, "gate_24": {"Gate_Type": "U3Gate", "Angle_3": 1.0, "Control": 3, "Angle_1": 0.5, "Target": -1, "Angle_2": 0.0, "Gate_Number": 24}, "gate_13": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 1, "Angle_1": -1, "Target": 0, "Angle_2": -1, "Gate_Number": 13}, "gate_14": {"Gate_Type": "U3Gate", "Angle_3": 0.5, "Control": 4, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 14}, "gate_26": {"Gate_Type": "Measure", "Angle_3": -1, "Control": 0, "Angle_1": -1, "Target": -1, "Angle_2": -1, "Gate_Number": 26}, "gate_07": {"Gate_Type": "U3Gate", "Angle_3": 1.0, "Control": 3, "Angle_1": 0.5, "Target": -1, "Angle_2": 0.0, "Gate_Number": 7}, "gate_03": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 2, "Angle_1": -1, "Target": 3, "Angle_2": -1, "Gate_Number": 3}, "gate_21": {"Gate_Type": "U3Gate", "Angle_3": -0.5, "Control": 4, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 21}, "gate_22": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 4, "Angle_1": -1, "Target": 3, "Angle_2": -1, "Gate_Number": 22}, "gate_36": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_04": {"Gate_Type": "U3Gate", "Angle_3": 0.5, "Control": 3, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 4}},
{"statevector": {"state_11111": 0, "state_10111": 0, "state_11011": 0, "state_11100": 0, "state_01001": 1024, "state_01100": 0, "state_00000": 0, "state_00100": 0, "state_01000": 0, "state_11110": 0, "state_11000": 0, "state_00001": 0, "state_00111": 0, "state_00010": 0, "state_11010": 0, "state_10000": 0, "state_10110": 0, "state_10010": 0, "state_00101": 0, "state_10100": 0, "state_01111": 0, "state_11001": 0, "state_10001": 0, "state_00011": 0, "state_10101": 0, "state_01010": 0, "state_11101": 0, "state_10011": 0, "state_01110": 0, "state_01101": 0, "state_00110": 0, "state_01011": 0}, "gate_11": {"Gate_Type": "U3Gate", "Angle_3": 0.5633414972337775, "Control": 3, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 11}, "gate_30": {"Gate_Type": "Measure", "Angle_3": -1, "Control": 3, "Angle_1": -1, "Target": -1, "Angle_2": -1, "Gate_Number": 30}, "gate_28": {"Gate_Type": "Measure", "Angle_3": -1, "Control": 1, "Angle_1": -1, "Target": -1, "Angle_2": -1, "Gate_Number": 28}, "gate_08": {"Gate_Type": "U3Gate", "Angle_3": 0.5, "Control": 3, "Angle_1": 1.0, "Target": -1, "Angle_2": 0.5, "Gate_Number": 8}, "gate_12": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 0, "Angle_1": -1, "Target": 3, "Angle_2": -1, "Gate_Number": 12}, "gate_20": {"Gate_Type": "U3Gate", "Angle_3": 0.5, "Control": 2, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 20}, "gate_15": {"Gate_Type": "U3Gate", "Angle_3": -0.5, "Control": 4, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 15}, "gate_32": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_23": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 2, "Angle_1": -1, "Target": 4, "Angle_2": -1, "Gate_Number": 23}, "gate_02": {"Gate_Type": "U3Gate", "Angle_3": 0.25, "Control": 0, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 2}, "gate_37": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_31": {"Gate_Type": "Measure", "Angle_3": -1, "Control": 4, "Angle_1": -1, "Target": -1, "Angle_2": -1, "Gate_Number": 31}, "gate_10": {"Gate_Type": "U3Gate", "Angle_3": 0.25, "Control": 1, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 10}, "gate_17": {"Gate_Type": "U3Gate", "Angle_3": -0.5, "Control": 2, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 17}, "gate_01": {"Gate_Type": "U3Gate", "Angle_3": 1.0, "Control": 0, "Angle_1": 0.5, "Target": -1, "Angle_2": 0.0, "Gate_Number": 1}, "gate_39": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_00": {"Gate_Type": "U3Gate", "Angle_3": 0.5, "Control": 0, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 0}, "gate_40": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_05": {"Gate_Type": "U3Gate", "Angle_3": 1.0, "Control": 0, "Angle_1": 0.5, "Target": -1, "Angle_2": 0.0, "Gate_Number": 5}, "gate_16": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 2, "Angle_1": -1, "Target": 4, "Angle_2": -1, "Gate_Number": 16}, "gate_18": {"Gate_Type": "U3Gate", "Angle_3": 0.5, "Control": 4, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 18}, "gate_27": {"Gate_Type": "Measure", "Angle_3": -1, "Control": 0, "Angle_1": -1, "Target": -1, "Angle_2": -1, "Gate_Number": 27}, "gate_25": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 2, "Angle_1": -1, "Target": 4, "Angle_2": -1, "Gate_Number": 25}, "gate_34": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_35": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_06": {"Gate_Type": "U3Gate", "Angle_3": -0.5, "Control": 0, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 6}, "gate_29": {"Gate_Type": "Measure", "Angle_3": -1, "Control": 2, "Angle_1": -1, "Target": -1, "Angle_2": -1, "Gate_Number": 29}, "gate_38": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_19": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 4, "Angle_1": -1, "Target": 2, "Angle_2": -1, "Gate_Number": 19}, "QASM_File_Name": "3125_random_5_qubit_circuits/circuit_1746.qasm", "gate_33": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_09": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 1, "Angle_1": -1, "Target": 3, "Angle_2": -1, "Gate_Number": 9}, "gate_24": {"Gate_Type": "U3Gate", "Angle_3": -1.0225828058102693, "Control": 4, "Angle_1": -0.47730070348744585, "Target": -1, "Angle_2": 0.0, "Gate_Number": 24}, "gate_13": {"Gate_Type": "U3Gate", "Angle_3": -0.5633414972337775, "Control": 3, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 13}, "gate_14": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 0, "Angle_1": -1, "Target": 3, "Angle_2": -1, "Gate_Number": 14}, "gate_26": {"Gate_Type": "U3Gate", "Angle_3": 0.0, "Control": 4, "Angle_1": 0.47730070348744585, "Target": -1, "Angle_2": 1.7251934917156755, "Gate_Number": 26}, "gate_07": {"Gate_Type": "U3Gate", "Angle_3": 0.5, "Control": 0, "Angle_1": 1.0, "Target": -1, "Angle_2": 0.5, "Gate_Number": 7}, "gate_03": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 1, "Angle_1": -1, "Target": 0, "Angle_2": -1, "Gate_Number": 3}, "gate_21": {"Gate_Type": "U3Gate", "Angle_3": 1.0225828058102693, "Control": 2, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 21}, "gate_22": {"Gate_Type": "U3Gate", "Angle_3": -0.7026106859054063, "Control": 4, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 22}, "gate_36": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_04": {"Gate_Type": "U3Gate", "Angle_3": -0.25, "Control": 0, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 4}},
{"statevector": {"state_11111": 0, "state_10111": 0, "state_11011": 0, "state_11100": 0, "state_01001": 0, "state_01100": 0, "state_00000": 516, "state_00100": 0, "state_01000": 508, "state_11110": 0, "state_11000": 0, "state_00001": 0, "state_00111": 0, "state_00010": 0, "state_11010": 0, "state_10000": 0, "state_10110": 0, "state_10010": 0, "state_00101": 0, "state_10100": 0, "state_01111": 0, "state_11001": 0, "state_10001": 0, "state_00011": 0, "state_10101": 0, "state_01010": 0, "state_11101": 0, "state_10011": 0, "state_01110": 0, "state_01101": 0, "state_00110": 0, "state_01011": 0}, "gate_11": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_30": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_28": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_08": {"Gate_Type": "Measure", "Angle_3": -1, "Control": 2, "Angle_1": -1, "Target": -1, "Angle_2": -1, "Gate_Number": 8}, "gate_12": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_20": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_15": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_32": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_23": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_02": {"Gate_Type": "U3Gate", "Angle_3": 1.0, "Control": 3, "Angle_1": 0.5, "Target": -1, "Angle_2": 0.0, "Gate_Number": 2}, "gate_37": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_31": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_10": {"Gate_Type": "Measure", "Angle_3": -1, "Control": 4, "Angle_1": -1, "Target": -1, "Angle_2": -1, "Gate_Number": 10}, "gate_17": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_01": {"Gate_Type": "U3Gate", "Angle_3": 1.6262791423758534, "Control": 2, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 1}, "gate_39": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_00": {"Gate_Type": "U3Gate", "Angle_3": 1.0, "Control": 0, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 0}, "gate_40": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_05": {"Gate_Type": "U3Gate", "Angle_3": 0.5, "Control": 4, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 5}, "gate_16": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_18": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_27": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_25": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_34": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_35": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_06": {"Gate_Type": "Measure", "Angle_3": -1, "Control": 0, "Angle_1": -1, "Target": -1, "Angle_2": -1, "Gate_Number": 6}, "gate_29": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_38": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_19": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "QASM_File_Name": "3125_random_5_qubit_circuits/circuit_220.qasm", "gate_33": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_09": {"Gate_Type": "Measure", "Angle_3": -1, "Control": 3, "Angle_1": -1, "Target": -1, "Angle_2": -1, "Gate_Number": 9}, "gate_24": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_13": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_14": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_26": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_07": {"Gate_Type": "Measure", "Angle_3": -1, "Control": 1, "Angle_1": -1, "Target": -1, "Angle_2": -1, "Gate_Number": 7}, "gate_03": {"Gate_Type": "U3Gate", "Angle_3": -0.5, "Control": 4, "Angle_1": 0.0, "Target": -1, "Angle_2": 0.0, "Gate_Number": 3}, "gate_21": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_22": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_36": {"Gate_Type": "", "Angle_3": -1, "Control": -1, "Angle_2": -1, "Gate_Number": -1, "Target": -1, "Angle_1": -1}, "gate_04": {"Gate_Type": "CnotGate", "Angle_3": -1, "Control": 1, "Angle_1": -1, "Target": 4, "Angle_2": -1, "Gate_Number": 4}}]

Writing 10_quantum_circuits.json


In [0]:
import json
with open('10_quantum_circuits.json') as json_file:
    data2 = json.load(json_file)
print(len(data2)) # prints 10

10


We then pre-process the data so that it can be use by our neural network.

In [0]:
# PLACE PROVIDED INPUT DATA INTO DATA FRAME
def x_data_preprocess(data):
  """
  to construct a one flat array for the x input data
  reflecting all the gates, along with angles, etc.
  we first construct an empty dataframe with needed columns
  representing all the gates and angles
  then we fetch data row by row and populate the dataframe.
  because we fetch the data by the same order as the dataframe, we can input 
  the data, in that order into the dataframe.
  """
 
  df = pd.DataFrame()
  keys_idx = [idx for idx in sorted(data[0].keys()) if 'gate' in idx]
  label_list = ['Angle_1', 'Angle_2', 'Angle_3', 'Control', 'Target']

  for i in keys_idx:
    for j in label_list:
      col_idx = str(i)+'_'+str(j)
      df[col_idx] = []

  for h in range(len(data)):
    tmp = [data[h][i][j] for i in keys_idx for j in label_list]
    df = df.append(pd.Series(tmp, index=df.columns), ignore_index=True)

  # number of features
  print(len(df.columns))
  
  # number of samples
  print(len(df))
  
  # output will have 32 nodes
  print(len(data[0]['statevector']))

  return df

In [0]:
def y_data_preprocess(data):
  """
  we first scale the y target data such that we have a probability distribution.
  then, we populate the dataframe with each row representing a sample target distribution.
  """
  # PLACE PROVIDED TARGET DATA INTO DATA FRAME
  # Scale into distribution with sum of probabilities equal to 1.0
  # https://stackoverflow.com/questions/30964577/divide-each-python-dictionary-value-by-total-value
  y_data = {}
  for i in range(len(data)):
    tot = sum(data[i]['statevector'].values())
    y_data[i] = {k: v / tot for k, v in data[i]['statevector'].items()}

  # CONSTRUCT DATAFRAME TO HOLD Y DATA.
  # EACH ROW REPRESENT A SAMPLE
  # EACH SAMPLE IS A VECTOR, REPRESENTING DISTRUBUTION
  df_y = pd.DataFrame(columns=[k for k in data[0]['statevector'].keys()])
  for i in range(len(data)):
    df_y.loc[i] = [v for v in y_data[i].values()]

  return df_y

In [0]:
def data_split(data, df, df_y, train_split=0.8, val_split=0.9):
  """
  data: original raw data provided
  df: dataframe for x input data
  df_y: dataframe for y target data
  train_split: 0.9 represent 90%
  val_split: 0.95 represent 5% for validation data
  we split the data into train, validation and testing.
  the user specifies the split as arguments to the function.
  """
  # split data into train, val, and test.
  m = len(data)
  x_train = df.iloc[0 : int(train_split * m), :]
  y_train = df_y.iloc[0 : int(train_split * m), :].values
  x_val = df.iloc[int(train_split * m + 1) : int(val_split * m), :]
  y_val = df_y.iloc[int(train_split * m + 1) : int(val_split * m), :].values
  x_test = df.iloc[int(val_split * m + 1) : m, :]
  y_test = df_y.iloc[int(val_split * m + 1) : m, :].values

  print(x_train.shape)
  print(y_train.shape)
  print(x_val.shape)
  print(y_val.shape)
  print(x_test.shape)
  print(y_test.shape)
  return (x_train, y_train, x_val, y_val, x_test, y_test)

In [0]:
import pandas as pd

In [0]:
df = x_data_preprocess(data2)
df_y = y_data_preprocess(data2)
(x_train, y_train, x_val, y_val, x_test, y_test) = data_split(data2, df, df_y, train_split=0.7, val_split=0.9)

205
10
32
(3, 205)
(3, 32)
(0, 205)
(0, 32)
(6, 205)
(6, 32)


In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

# Load the TensorBoard notebook extension.
%load_ext tensorboard

TensorFlow 2.x selected.


In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow import keras

import numpy as np
import pandas as pd

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

TensorFlow version:  2.0.0


In [0]:
from tensorflow.keras import layers

model = tf.keras.Sequential()
model.add(layers.Dense(102, activation='relu', input_shape=(205,)))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(32, activation='softmax'))

Now we compile the model

In [0]:
# Compile the model.
model.compile(
  optimizer='adam',
  loss='categorical_crossentropy'
  #metrics=['accuracy'],
)

In [0]:
# Clear any logs from previous runs
!rm -rf ./logs/ 

In [0]:
# SET THE LOG FILE NAME AND THE CALLBACK
logdir = 'logs/' + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=50, write_graph=True, write_images=True)

Train the model

In [0]:
EPOCHS=70

# Train the model.
history = model.fit(
  x_train,
  y_train,
  epochs=EPOCHS,
  batch_size=100,
  verbose=0,
  #validation_split=0.1,
  validation_data=(x_val, y_val), # validation data used to monitor validation loss at the end of each epoch
  callbacks=[tensorboard_callback],
)

Here we launch TensorBoard to see the progress as the model trains.

In [0]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 196), started 0:01:42 ago. (Use '!kill 196' to kill it.)

You can save the model weights for future reference

In [0]:
model.save_weights('model.h5')

And you can also load the saved weights to predict statevectors for quantum circuits!

In [0]:
from keras.models import Sequential
from keras.layers import Dense
# use the architecture as above for the model.
# Load the model's saved weights.
model.load_weights('model.h5')

Using TensorFlow backend.


In [0]:
# Predict on select data
predictions = model.predict(x_test[:1])

# Print ground truth
print(y_test[0])
print('\n')
# Print our model's predictions.
print(predictions)

[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]


[[6.2798807e-07 8.4055733e-07 4.3597727e-05 2.1840318e-04 1.7675817e-03
  3.1319435e-06 5.2903146e-01 8.1587298e-04 2.2162017e-03 1.3141071e-03
  1.3854662e-05 4.6528038e-07 7.3993988e-06 4.4741569e-07 8.6083543e-03
  1.3782032e-01 4.5915465e-03 3.7050608e-08 4.1279446e-08 4.3890986e-04
  4.1501644e-06 5.6648895e-07 1.3083605e-04 4.5170427e-07 1.0051192e-07
  6.3324109e-02 4.3048974e-04 1.8433859e-03 9.4183834e-09 1.0644725e-01
  1.3799413e-01 2.9312454e-03]]
